# Optimized Sphere Strings

In [1]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import tensorflow as tf
import random

from tqdm.notebook import tqdm

from loss_functions import ProbabilityMSE, channel_fidelity_loss
from optimization import ModelQuantumMap, Logger
from kraus_channels import KrausMap, DilutedKrausMap
from lindblad_channels import CompactLindbladMap
from spam import InitialState, POVM, SPAM, CorruptionMatrix, IdealSPAM
from utils import saver
from synthetic_data import generate_spam_data, generate_spam_benchmark, generate_map_data
from quantum_tools import add_shot_noise, apply_unitary, measurement, generate_unitary
from set_precision import precision
from experimental import generate_pauli_circuits
from string_inputs import HaarStrings

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## Recover Map

In [2]:
def generate_map_data(channel_target, inputs_map, shots = 1024):
    d = channel_target.d
    U_prep, U_basis = inputs_map

    N_map = U_prep.shape[0]
    init = np.zeros((d,d))
    init[0,0] = 1
    init = tf.cast(init, dtype=precision)
    
    state = tf.repeat(tf.expand_dims(init, axis=0), N_map, axis=0)
    state = apply_unitary(state, U_prep)
    state = channel_target.apply_channel(state)
    targets_map = measurement(state, U_basis)
    
    #add noise
    targets_map = add_shot_noise(targets_map, shots=shots)
    
    return targets_map

In [3]:
n = 3
d = 2**n

N = 2000

#Pauli Strings
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

inputs_map1, _ = generate_pauli_circuits(n=n, N=N)

#Haar Strings
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

strings = HaarStrings(N=2*N, n=n)

_, U_haar = strings.generate_circuits()
inputs_map2 = [U_haar[:N], U_haar[N:]]

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

channel_target = DilutedKrausMap(U = generate_unitary(d=d), c=0.5, kraus_part = KrausMap(d=d, rank=3))
targets_map1 = generate_map_data(channel_target, inputs_map1, shots=1024)
targets_map2 = generate_map_data(channel_target, inputs_map2, shots=1024)

### Vanilla Pauli Strings

In [4]:
np.random.seed(43)
random.seed(43)
tf.random.set_seed(43)
model_list = [[]]
for i, rank in enumerate([4, 8, 16, 32, 64]):
    print(rank)
    model = ModelQuantumMap(channel = KrausMap(d=d, 
                                            rank=rank,),
                                loss_function = ProbabilityMSE(),
                                optimizer = tf.optimizers.Adam(learning_rate=0.01),
                                logger = Logger(loss_function = ProbabilityMSE(),
                                                loss_function_val = ProbabilityMSE()),
                                )
    for _ in range(4):
        model.train(inputs = inputs_map1,
                    targets = targets_map1,
                    inputs_val = inputs_map2,
                    targets_val = targets_map2,
                    num_iter = 1000,
                    N = 500,
                    verbose=0,
                    )
        
        model_list[i].append(model)

4
0.006448724792290887 0.0068300596035372006
0.0064286092288114195 0.006828145827639948
0.006451340951727526 0.0068622474946732825
0.006405769677670529 0.006904431001970069
8
0.006222114135295119 0.007014478348111685


IndexError: list index out of range

In [28]:
np.random.seed(43)
random.seed(43)
tf.random.set_seed(43)
model_list = []
for rank in tqdm([4, 8, 16, 32, 64]):
    model = ModelQuantumMap(channel = KrausMap(d=d, 
                                            rank=rank,),
                                loss_function = ProbabilityMSE(),
                                optimizer = tf.optimizers.Adam(learning_rate=0.01),
                                logger = Logger(loss_function = ProbabilityMSE(),
                                                loss_function_val = channel_fidelity_loss),
                                )

    model.train(inputs = inputs_map1,
                targets = targets_map1,
                inputs_val = None,
                targets_val = [kraus_target],
                num_iter = 4000,
                N = 500,
                verbose=0,
                )
    
    model_list.append(model)

  0%|          | 0/5 [00:00<?, ?it/s]

0.006574786622024594 -0.999239242214913
0.006356087668005296 -0.9767154889989185
0.006234943154259028 -0.9717255190249705
0.006175880754871784 -0.9718413178379592


KeyboardInterrupt: 